In [1]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import gc
import cell2location
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy.sparse as sparse
from scvi.data import synthetic_iid
from scvi.dataloaders import AnnDataLoader

import diopy
from cell2location import run_colocation
from cell2location.models import Cell2location, RegressionModel
import cell2location
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import scvi

from cell2location import run_colocation
from cell2location.models import Cell2location, RegressionModel


import torch
import scvi


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Global seed set to 0
D:\anaconda\envs\cell2loc_env\lib\site-packages\pytorch_lightning-1.7.7-py3.9.egg\pytorch_lightning\utilities\warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
D:\anaconda\envs\cell2loc_env\lib\site-packages\pytorch_lightning-1.7.7-py3.9.egg\pytorch_lightning\utilities\warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
#读取文件
import os
import pandas as pd



# 设置工作目录
working_dir = 'E:/doctor/自研/mapping/data/20230814final/h5/st/'
os.chdir(working_dir)

# 获取所有文件列表
datalist = [f for f in os.listdir() if os.path.isfile(f)]
datalist

['C2D2_CB_cellbin.h5',
 'C2D2_HIP_cellbin.h5',
 'C2D2_OB_cellbin.h5',
 'HIP_slide_seq.h5',
 'HIP_test1.h5',
 'st_HIP_starmap.h5',
 'st_human_CTX.h5',
 'st_mouse_CTX.h5']

In [6]:
sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_CB.h5")
sc_obj

AnnData object with n_obs × n_vars = 3573 × 28526
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rb', 'percent.HB', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.3', 'seurat_clusters', 'annotation', 'type', 'number', 'SCT_snn_res.0.1', 'umapx', 'umapy', 'cut', 'SCT_snn_res.0.2'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [13]:
for i in range(0, 2):
    print(datalist[i])

C2D2_CB_cellbin.h5
C2D2_HIP_cellbin.h5


In [15]:
st_path

'E:/doctor/自研/mapping/data/20230814final/st/C2D2_CB_cellbin.rds'

In [ ]:
import time

# 设置工作目录
working_dir = 'E:/doctor/自研/mapping/data/20230814final/h5/st/'
os.chdir(working_dir)

# 获取所有文件列表
datalist = [f for f in os.listdir() if os.path.isfile(f)]

# for i in range(0, len(datalist)):
for i in range(3, 4):
#     if(datalist[i]!="C2D2_HIP_cellbin.h5"):
#         continue
    print(datalist[i])

    # 读取st数据
    st_path = os.path.join(working_dir, datalist[i])
    st_obj = diopy.input.read_h5(st_path)
    
    # 根据st数据选取sc数据
    if datalist[i] in ["st_HIP_starmap.h5", "C2D2_HIP_cellbin.h5", "HIP_slide_seq.h5", "HIP_test1.h5"]:
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_HIP.h5")
        anno = 'annotation'

    elif datalist[i] == "C2D2_CB_cellbin.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_CB.h5")
        anno = 'annotation'

    elif datalist[i] == "C2D2_OB_cellbin.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_OB.h5")
        anno = 'type'

    elif datalist[i] == "st_human_CTX.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_human_CTX.h5")
        anno = 'Subclass'

    elif datalist[i] == "st_mouse_CTX.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_mouse_CTX.h5")
        anno = 'Type'
    #开始时间
    start_time = time.time()
    
    adata_ref=sc_obj
    adata_ref.X=sparse.csr_matrix(np.round(adata_ref.X.todense()))
    RegressionModel.setup_anndata(adata_ref, labels_key=anno)
    mod = RegressionModel(adata_ref)
    mod.train(max_epochs=250, batch_size=2000, train_size=1, lr=0.002
              #, use_gpu=True
             )
    adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 1500
    #, 'use_gpu': True
    }
    )
    if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
        inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                        for i in adata_ref.uns['mod']['factor_names']]].copy()
    else:
        inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                        for i in adata_ref.uns['mod']['factor_names']]].copy()
    inf_aver.columns = adata_ref.uns['mod']['factor_names']
    adata_vis=st_obj
    adata_vis.X=sparse.csr_matrix(np.round(adata_vis.X.todense()))
    intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
    adata_vis = adata_vis[:, intersect].copy()
    inf_aver = inf_aver.loc[intersect, :].copy()
    Cell2location.setup_anndata(adata=adata_vis)
    mod = cell2location.models.Cell2location(
        adata_vis, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
        N_cells_per_location=1.3,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection (using default here):
        detection_alpha=200
    )
    mod.train(max_epochs=250,
          # train using full data (batch_size=None)
          batch_size=500,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1
          #,use_gpu=True
          )
    adata_vis = mod.export_posterior(
    adata_vis, sample_kwargs={'num_samples': 250, 'batch_size': mod.adata.n_obs#, 'use_gpu': True
        }
    )

    print("saving \n")
    #重写一个名称
    name="E:/doctor/自研/mapping/data/20230814final/result/cell2/"+datalist[i]+"_cell2_prediction.csv"
    adata_vis.obsm["means_cell_abundance_w_sf"].to_csv(name)
    
#     end_time = time.time()
#     time_diff = end_time-start_time
#     print(time_diff)
    
#     time_name="E:/doctor/自研/mapping/data/20230814final/result/cell2location/"+datalist[i]+"_cell2_prediction_time.csv"
#     pd.Series([time_diff], name='time_diff').to_csv(time_name, header=True)

HIP_slide_seq.h5


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/250:   2%|▋                                       | 4/250 [00:31<31:44,  7.74s/it, v_num=1, elbo_train=3.77e+8]

In [5]:
name="E:/doctor/自研/mapping/data/20230814final/result/cell2/"+datalist[i]+"_cell2_prediction.csv"
adata_vis.obsm["means_cell_abundance_w_sf"].to_csv(name)

In [4]:
name

'E:/doctor/自研/mapping/data/20230814final/result/cell2location/C2D2_HIP_cellbin.h5_cell2_prediction.csv'